In [2]:
%load_ext autoreload
%autoreload 2

In [35]:
from pathlib import Path
from itertools import count

import cv2
import torch
import torchvision.transforms as T

from sigmaclast_data import SigmaclastFolder

In [19]:
root = Path("../data/original_data/")

# transform = T.Compose([T.ToTensor(), T.Resize((32, 32))])
transform = None

In [20]:
dataset = SigmaclastFolder(root, transform=transform)

In [21]:
img, label = dataset[3]

In [101]:
SigmaclastFolder?

Init signature:
SigmaclastFolder(
    root: str,
    transform: Optional[Callable] = None,
    target_transform: Optional[Callable] = None,
    loader: Callable[[str], Any] = <function default_loader at 0x168eb7dc0>,
    is_valid_file: Optional[Callable[[str], bool]] = None,
)
Docstring:     
A generic data loader where the images are arranged in this way by default: ::

    root/dog/xxx.png
    root/dog/xxy.png
    root/dog/[...]/xxz.png

    root/cat/123.png
    root/cat/nsdf3.png
    root/cat/[...]/asd932_.png

This class inherits from :class:`~torchvision.datasets.DatasetFolder` so
the same methods can be overridden to customize the dataset.

Args:
    root (string): Root directory path.
    transform (callable, optional): A function/transform that  takes in an PIL image
        and returns a transformed version. E.g, ``transforms.RandomCrop``
    target_transform (callable, optional): A function/transform that takes in the
        target and transforms it.
    loader (callable, op

In [51]:
cntr = count()

In [85]:
root.parts

('..', 'data', 'original_data')

In [60]:
dir(cntr)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [98]:
def preprocess_dataset(root):
    root = Path(root)
    original_dir = root/"original_data"
    CW_dir = root/"processed_data/CW"
    CCW_dir = root/"processed_data/CCW"
    CW_dir.mkdir(parents=True, exist_ok=True)
    CCW_dir.mkdir(parents=True, exist_ok=True)
    
    img_count = count()
    
    def preprocess_img(img_file, label):
        img = cv2.imread(str(img_file), cv2.IMREAD_COLOR)
        # `1` flips img horizonally.
        # This flips the label as well.
        # TODO: add explanation why...
        img_flipped = cv2.flip(img, 1)
        img_num = next(img_count)
        name = f"img_{img_num}_{label}.jpg"
        name_flipped = f"img_{img_num}_{label}.jpg"
        
        try:
            if label == "CW":
                cv2.imwrite(str(CW_dir/name), img)
                cv2.imwrite(str(CCW_dir/name_flipped), img_flipped)
            else:
                cv2.imwrite(str(CCW_dir/name), img)
                cv2.imwrite(str(CW_dir/name_flipped), img_flipped)
        except cv2.error as e:
            print(f"Error: {e}")
            print(f"Continuing...")
            

    for directory in original_dir.iterdir():
        label = directory.parts[-1]
        if label == "CW" or label == "CCW":
            for img_file in directory.iterdir():
                preprocess_img(img_file, label)

In [100]:
preprocess_dataset("../data")

Error: OpenCV(4.5.5) /Users/runner/miniforge3/conda-bld/libopencv_1648505774709/work/modules/imgcodecs/src/loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'

Continuing...
